In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision torchaudio
!pip install tqdm matplotlib scikit-learn

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from PIL import Image

In [ ]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of the current GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Current GPU: {gpu_name}")

    # Get GPU device properties
    device = torch.device('cuda')
    print(f"Device type: {device.type}")
else:
    print("CUDA is not available. PyTorch will run on the CPU.")

Number of GPUs available: 1
Current GPU: Tesla T4
Device type: cuda


In [ ]:
# Standard image size for CNNs
image_size = 224

# Custom dataset class to handle corrupt images
class CustomImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except Exception as e:
            print(f"Skipping corrupted image: {self.imgs[index][0]} - {e}")
            return self.__getitem__((index + 1) % len(self.imgs))  # Get next valid image

# Optimized Transformations (SAME FOR ALL MODELS)
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Lambda(lambda img: img.convert('RGB')),  # Ensures all images are RGB
    transforms.RandomHorizontalFlip(p=0.5),  # Flip 50% of the time
    transforms.RandomRotation(10),  # Rotate within ±10 degrees
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),  # Random cropping
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Color variation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Paths
dataset_path = "/content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset"
validation_path = "/content/drive/MyDrive/Waste Classification using Transfer Learning/Data Validation"

# Hyperparameters (Optimized for Performance)
batch_size = 32  # Changed from 64 to 32
num_workers = 4  # Parallel data loading

# Load dataset with corrupt image handling
train_dataset = CustomImageFolder(root=dataset_path, transform=train_transforms)
val_dataset = CustomImageFolder(root=validation_path, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Print class names to verify
print("Classes:", train_dataset.classes)

# Function to check for corrupt images before training
def check_corrupt_images(dataset_path):
    print("Checking dataset for corrupt images...")
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # Verify image integrity
                except:
                    print(f"Corrupt or non-image file found: {file_path}")

# Run dataset check
check_corrupt_images(dataset_path)
check_corrupt_images(validation_path)

print("Dataset loaded successfully!")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classes: ['cardboard', 'e-waste', 'glass', 'medical', 'metal', 'paper', 'plastic']
Checking dataset for corrupt images...
Corrupt or non-image file found: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/paper/paper 2273.jpg
Corrupt or non-image file found: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/cardboard/cardboard 1075.jpg
Corrupt or non-image file found: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/e-waste/e-waste 1719.jpg
Checking dataset for corrupt images...
Dataset loaded successfully!


In [ ]:
def train_model(model, model_name, criterion, optimizer, train_loader, val_loader, device, num_epochs=25, patience=5, scheduler=None):
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    best_val_acc = 0  # Track best validation accuracy
    best_val_loss = float('inf')  # Track best validation loss
    patience_counter = 0

    # Create directory for saving models
    save_dir = os.path.join("/content/drive/MyDrive/Waste Classification using Transfer Learning/Saved Models", model_name)
    os.makedirs(save_dir, exist_ok=True)

    print("Starting training...")

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs} started...")
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx}/{len(train_loader)}: Loss = {loss.item():.4f}")

        train_loss = running_loss / len(train_loader)
        train_acc = (correct / total) * 100

        print("Training phase completed. Starting validation...")

        # Validation Phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = (val_correct / val_total) * 100

        # Compute Precision, Recall, and F1-score
        precision = precision_score(all_labels, all_preds, average='macro') * 100
        recall = recall_score(all_labels, all_preds, average='macro') * 100
        f1 = f1_score(all_labels, all_preds, average='macro') * 100

        # Store metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        # Print Metrics for Each Epoch
        print(f"Epoch [{epoch}/{num_epochs}] completed.")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"Precision: {precision:.2f}% | Recall: {recall:.2f}% | F1-score: {f1:.2f}%")

        # Save checkpoint every 5 epochs
        if epoch % 5 == 0:
            checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch_{epoch}.pth')
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'val_acc': val_acc
            }
            torch.save(checkpoint, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}!")

        # Save best model (based on validation accuracy or loss)
        if val_loss < best_val_loss or val_acc > best_val_acc:
            best_val_loss = min(best_val_loss, val_loss)
            best_val_acc = max(best_val_acc, val_acc)
            patience_counter = 0  # Reset patience counter

            best_model_path = os.path.join(save_dir, 'Model_MobileNetV2.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'val_acc': val_acc
            }, best_model_path)

            print(f"New best model saved at {best_model_path}!")
        else:
            patience_counter += 1  # Increment counter if no improvement

        # Early Stopping Condition
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch} due to no improvement in validation loss or accuracy.")
            break

    print("Training complete!")
    return train_losses, val_losses, train_accs, val_accs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import (
    resnet50, ResNet50_Weights,
    resnet34, ResNet34_Weights,
    efficientnet_b0, EfficientNet_B0_Weights,
    efficientnet_b3, EfficientNet_B3_Weights,
    vgg16, VGG16_Weights,
    densenet121, DenseNet121_Weights,
    mobilenet_v2, MobileNet_V2_Weights,
    inception_v3, Inception_V3_Weights
)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Select the model
selected_model_name = "MobileNet-V2"  # Change this to train other models

# Define models
models_list = {
    "ResNet-50": resnet50(weights=ResNet50_Weights.IMAGENET1K_V1),
    "ResNet-34": resnet34(weights=ResNet34_Weights.IMAGENET1K_V1),
    "EfficientNet-B0": efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1),
    "EfficientNet-B3": efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1),
    "VGG16": vgg16(weights=VGG16_Weights.IMAGENET1K_V1),
    "DenseNet-121": densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1),
    "MobileNet-V2": mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1),
    "Inception-V3": inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
}

# Get the selected model
model = models_list[selected_model_name]
print(f"\nTraining {selected_model_name}...\n")

# Adjust the final layer for 7 classes based on the model type
if "resnet" in selected_model_name.lower():
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 7)
elif "efficientnet" in selected_model_name.lower():
    num_features = model.classifier[1].in_features if hasattr(model.classifier, "__getitem__") else model.classifier.in_features
    model.classifier = nn.Linear(num_features, 7)
elif "densenet" in selected_model_name.lower():
    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, 7)
elif "vgg" in selected_model_name.lower():
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, 7)
elif "mobilenet" in selected_model_name.lower():
    num_features = model.classifier[1].in_features if hasattr(model.classifier, "__getitem__") else model.classifier.in_features
    model.classifier = nn.Linear(num_features, 7)
elif "inception" in selected_model_name.lower():
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 7)
    if model.aux_logits:  # Inception has an auxiliary classifier
        num_features_aux = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = nn.Linear(num_features_aux, 7)

# Move model to the device (CPU/GPU)
model = model.to(device)
model.train()  # Ensure the model is in training mode

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Train the model
train_losses, val_losses, train_accs, val_accs = train_model(
    model=model,
    model_name=selected_model_name,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    num_epochs=25,
    patience=5,
    scheduler=scheduler
)


Training MobileNet-V2...

Starting training...

Epoch 1/25 started...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Batch 0/455: Loss = 1.9737
Batch 10/455: Loss = 1.5207


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Batch 20/455: Loss = 1.3587
Batch 30/455: Loss = 1.0523


...

Batch 160/455: Loss = 0.0439
Batch 170/455: Loss = 0.0108
Batch 180/455: Loss = 0.0100
Batch 190/455: Loss = 0.1709
Batch 200/455: Loss = 0.0145


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Batch 210/455: Loss = 0.1636
Batch 220/455: Loss = 0.0223
Batch 230/455: Loss = 0.0249
Batch 240/455: Loss = 0.3261
Batch 250/455: Loss = 0.0554
Batch 260/455: Loss = 0.1316
Batch 270/455: Loss = 0.0323
Batch 280/455: Loss = 0.0922
Batch 290/455: Loss = 0.1719
Batch 300/455: Loss = 0.0274
Batch 310/455: Loss = 0.0085
Batch 320/455: Loss = 0.0520
Skipping corrupted image: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/e-waste/e-waste 1719.jpg - cannot identify image file <_io.BufferedReader name='/content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/e-waste/e-waste 1719.jpg'>
Batch 330/455: Loss = 0.0780
Batch 340/455: Loss = 0.0369
Batch 350/455: Loss = 0.0162
Batch 360/455: Loss = 0.0400
Batch 370/455: Loss = 0.1217
Batch 380/455: Loss = 0.0520
Batch 390/455: Loss = 0.0195
Batch 400/455: Loss = 0.1455
Batch 410/455: Loss = 0.0268
Batch 420/455: Loss = 0.0135
Batch 430/455: Loss = 0.2300
Batch 440/455: Loss = 0.0225
Batch 450/455: Loss

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [15/25] completed.
Train Loss: 0.0731 | Train Acc: 97.53%
Val Loss: 0.4084 | Val Acc: 89.83%
Precision: 90.34% | Recall: 89.50% | F1-score: 89.79%
Checkpoint saved at /content/drive/MyDrive/Waste Classification using Transfer Learning/Saved Models/MobileNet-V2/checkpoint_epoch_15.pth!
Early stopping at epoch 15 due to no improvement in validation loss or accuracy.
Training complete!


In [ ]:
import torch
# Define the folder path
save_path = "/content/drive/MyDrive/Waste Classification using Transfer Learning/Saved Models"
# os.makedirs(save_path, exist_ok=True)  # Create folder if it doesn't exist
torch.save(model, os.path.join(save_path, "Model_MobileNet-V2.pth"))
print(f"Model saved successfully in {save_path}/Model_MobileNet-V2.pth")

NameError: name 'model' is not defined